**二、Autograph 编码规范解析**

1，被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数。

In [2]:
import numpy as np
import tensorflow as tf

@tf.function
def np_random():
    a = np.random.randn(3, 3)
    tf.print(a)

@tf.function
def tf_random():
    a = tf.random.normal((3, 3))
    tf.print(a)

In [3]:
#np_random每次执行都是一样的结果。
np_random()
np_random()

array([[-0.74147446,  0.02272686, -1.68799382],
       [ 1.25273082, -0.19600605,  0.55517137],
       [ 0.61911347,  0.76474877,  0.80281486]])
array([[-0.74147446,  0.02272686, -1.68799382],
       [ 1.25273082, -0.19600605,  0.55517137],
       [ 0.61911347,  0.76474877,  0.80281486]])


In [4]:
#tf_random每次执行都会有重新生成随机数。
tf_random()
tf_random()

[[-1.78259957 -1.32271802 0.361095369]
 [-0.695781827 0.395475268 1.0455786]
 [1.63477051 -1.41514432 0.287649274]]
[[-0.58198452 0.0279216096 -0.928096175]
 [-0.681594849 -1.02384984 -1.22003853]
 [0.96960175 -1.00373185 0.0429180935]]


2，避免在@tf.function修饰的函数内部定义tf.Variable.

In [5]:
x = tf.Variable(1.0, dtype=tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return (x)

outer_var()
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [6]:
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)

#执行将报错
# inner_var()
# inner_var()

3,被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等结构类型变量。

In [7]:
tensor_list = []

#@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [8]:
tensor_list = []

@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list


append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)


[<tf.Tensor 'x:0' shape=() dtype=float32>]
